In [1]:
import pandas as pd
from gurobipy import *
import csv

Create dictionary where key = players, value = [calculated score, salary, PG, SG, SF, PF, C]

In [2]:
nbadict = {}
nbaplayers = []

firstline = True

with open("nbaplayerstats.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if firstline:
            firstline = False
            continue
            
        name = row[1]
        
        # create list of nba players
        nbaplayers.append(row[1])
        
        # dictionary entry - player: [calculated score, salary, PG, SG, SF, PF, C]
        calc_score = int(row[17])
        salary = int(float((row[9])))
        PG = int(row[18])
        SG = int(row[19])
        SF = int(row[20])
        PF = int(row[21])
        C = int(row[22])
        nbadict[name] = [calc_score, salary, PG, SG, SF, PF, C]

Create the model

In [3]:
m = Model()

Using license file /Users/angela/gurobi.lic
Academic license - for non-commercial use only


Create binary variables to determine if a player is selected on the team or not

In [4]:
x = m.addVars(nbaplayers, 
              lb = 0,
              ub = 1,
              vtype = GRB.INTEGER)

Create the constraints: 3 of each position (PG, SG, SF, PF, C)

In [5]:
# 3 PG's must be on the team
m.addConstr((quicksum(x[i]*nbadict[i][2] for i in nbaplayers) <= 3))

# 3 SG's must be on the team
m.addConstr((quicksum(x[i]*nbadict[i][3] for i in nbaplayers) <= 3))

# 3 SF's must be on the team
m.addConstr((quicksum(x[i]*nbadict[i][4] for i in nbaplayers) <= 3))

# 3 PF'S must be on the team
m.addConstr((quicksum(x[i]*nbadict[i][5] for i in nbaplayers) <= 3))

# 3 C's must be on the team
m.addConstr((quicksum(x[i]*nbadict[i][6] for i in nbaplayers) <= 3))

<gurobi.Constr *Awaiting Model Update*>

In [6]:
m.addConstr((quicksum(x[i] for i in nbaplayers) <= 15))

<gurobi.Constr *Awaiting Model Update*>

Create the constraint that all player salary must be less than the luxury tax threshold

In [7]:
luxtax = 132627000
m.addConstr((quicksum(x[i]*nbadict[i][1] for i in nbaplayers) <= luxtax))

<gurobi.Constr *Awaiting Model Update*>

Maximize overall team rating (sum of individual calculated scores)

In [8]:
m.setObjective(quicksum(x[i]*nbadict[i][0] for i in nbaplayers), GRB.MAXIMIZE)

In [9]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 7 rows, 267 columns and 798 nonzeros
Model fingerprint: 0xc122f71a
Variable types: 0 continuous, 267 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+07]
  Objective range  [7e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+08]
Found heuristic solution: objective 118.0000000
Presolve removed 0 rows and 12 columns
Presolve time: 0.00s
Presolved: 7 rows, 255 columns, 762 nonzeros
Variable types: 0 continuous, 255 integer (252 binary)

Root relaxation: objective 2.663656e+02, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  266.36558    0    2  118.00000  266.36558   126%     -    0s
H    0     0                     266.0000000  266.36558  0.14%     -    0s
     0     0  266.36558    0    2  266.00000  266.36558  0.14%  

Write output to .csv file

In [10]:
positions = {0: "PG", 1: "SG", 2: "SF", 3: "PF", 4: "C"} 

with open('out.csv', mode = 'w') as output:
    writer = csv.writer(output)
    writer.writerow(["Player", "Calculated Score", "Position", "Salary"])
    for i in nbaplayers:
        x1 = x[i].X
        if x1 == 1:
            index = 0
            for j in range(5):
                if nbadict[i][j+2] == 1:
                    index = j
            writer.writerow([i, nbadict[i][0], positions[index], nbadict[i][1]])

25842697
33526057
66268057
72541057
78933817
89533817
91179174
99179174
102708729
106162809
110929809
124970905
126261865
128209945
130709945
